# How to Build a Recommendation System for Purchase Data (Step-by-Step)
* Description: A documentation on building collaborative filtering models for recommending products to customers
* Link: https://medium.com/datadriveninvestor/how-to-build-a-recommendation-system-for-purchase-data-step-by-step-d6d7a78800b6
* Author: Moorissa Tjokro

## Problem statement
The steps below aim to recommend users their top 10 items to place into their basket. The final output will be a csv file in the `output` folder, and a function that searches for a recommendation list based on a speficied user:
* Input: user - customer ID
* Returns: ranked list of items (product IDs), that the user is most likely to want to put in his/her (empty) "basket"

## 1. Import modules
* `pandas` and `numpy` for data manipulation
* `turicreate` for performing model selection and evaluation
* `sklearn` for splitting the data into train and test set

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import time
import turicreate as tc
from sklearn.model_selection import train_test_split

import sys
sys.path.append("..")
import scripts.data_layer as data_layer

## 2. Load data
Two datasets are used in this exercise, which can be found in `data` folder: 
* `recommend_1.csv` consisting of a list of 1000 customer IDs to recommend as output
* `trx_data.csv` consisting of user transactions

The format is as follows.

In [2]:
customers = pd.read_csv('../data/recommend_1.csv')
transactions = pd.read_csv('../data/trx_data.csv')

In [3]:
print(customers.shape)
customers.head()
#print(customers) raakshasan21022019

(1000, 1)


,customerId
0,1553
1,20400
2,19750
3,6334
4,27773


In [4]:
print(transactions.shape)
transactions.head()

(62483, 2)


,customerId,products
0,0,20
1,1,2|2|23|68|68|111|29|86|107|152
2,2,111|107|29|11|11|11|33|23
3,3,164|227
4,5,2|2


## 3. Data preparation
* Our goal here is to break down each list of items in the `products` column into rows and count the number of products bought by a user

In [5]:
# example 1: split product items
transactions['products'] = transactions['products'].apply(lambda x: [int(i) for i in x.split('|')])
transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index()


,customerId,0,1,2,3,4,5,6,7,8,9
0,0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2.0,2.0,23.0,68.0,68.0,111.0,29.0,86.0,107.0,152.0


In [6]:
# example 2: organize a given table into a dataframe with customerId, single productId, and purchase count
pd.melt(transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})

,customerId,productId,purchase_count
0,0,20.0,1
1,1,2.0,2
2,1,23.0,1
3,1,29.0,1
4,1,68.0,2
5,1,86.0,1
6,1,107.0,1
7,1,111.0,1
8,1,152.0,1


### 3.1. Create data with user, item, and target field
* This table will be an input for our modeling later
    * In this case, our user is `customerId`, `productId`, and `purchase_count`

In [28]:
s=time.time()

data = pd.melt(transactions.set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})
data['productId'] = data['productId'].astype(np.int64)

print("Execution time:", round((time.time()-s)/60,2), "minutes")

Execution time: 0.32 minutes


In [29]:
print(data.shape)
print(data)
data.head()

(133585, 3)
        customerId  productId  purchase_count
0                0          1               2
1                0         13               1
2                0         19               3
3                0         20               1
4                0         31               2
5                0         52               1
6                0         69               2
7                0         93               3
8                0        136               2
9                0        157               1
10               0        198               1
11               0        216               1
12               0        255               2
13               0        256               1
14               0        260               5
15               1          2               6
16               1         23               1
17               1         25               1
18               1         29               1
19               1         61               2
20               1    

,customerId,productId,purchase_count
0,0,1,2
1,0,13,1
2,0,19,3
3,0,20,1
4,0,31,2


### 3.2. Create dummy
* Dummy for marking whether a customer bought that item or not.
* If one buys an item, then `purchase_dummy` are marked as 1
* Why create a dummy instead of normalizing it, you ask?
    * Normalizing the purchase count, say by each user, would not work because customers may have different buying frequency don't have the same taste
    * However, we can normalize items by purchase frequency across all users, which is done in section 3.3. below.

In [10]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy

In [11]:
data_dummy = create_data_dummy(data)

In [12]:
print(data_dummy.shape)
data_dummy.head()

(133585, 4)


,customerId,productId,purchase_count,purchase_dummy
0,0,1,2,1
1,0,13,1,1
2,0,19,3,1
3,0,20,1,1
4,0,31,2,1


### 3.3. Normalize item values across users
* To do this, we normalize purchase frequency of each item across users by first creating a user-item matrix as follows

In [13]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
df_matrix.head()

productId,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,
0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
(df_matrix.shape)

(24429, 300)

In [15]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
print(df_matrix_norm.shape)
df_matrix_norm.head()

(24429, 300)


productId,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,
0,NaN,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,0.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# create a table for input to the modeling

d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head()

(133585, 3)


,customerId,productId,scaled_purchase_freq
9,9,0,0.133333
25,25,0,0.133333
32,33,0,0.133333
35,36,0,0.133333
43,44,0,0.133333


#### Define a function for normalizing data

In [17]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()

* We can normalize the their purchase history, from 0-1 (with 1 being the most number of purchase for an item and 0 being 0 purchase count for that item).

## 4. Split train and test set
* Splitting the data into training and testing sets is an important part of evaluating predictive modeling, in this case a collaborative filtering model. Typically, we use a larger portion of the data for training and a smaller portion for testing. 
* We use 80:20 ratio for our train-test set size.
* Our training portion will be used to develop a predictive model, while the other to evaluate the model's performance.
* Now that we have three datasets with purchase counts, purchase dummy, and scaled purchase counts, we would like to split each.

In [18]:
train, test = train_test_split(data, test_size = .2)
print(train.shape, test.shape)

(106868, 3) (26717, 3)


In [19]:
# Using turicreate library, we convert dataframe to SFrame - this will be useful in the modeling part

train_data = tc.SFrame(train)
test_data = tc.SFrame(test)

In [20]:
train_data

customerId,productId,purchase_count
14719,14,1
581,12,1
331,104,1
5597,92,1
2581,241,3
5209,290,1
1034,229,2
968,54,1
7784,157,1
3172,9,2


In [21]:
test_data

customerId,productId,purchase_count
912,9,3
18874,66,1
1070,48,2
171,78,1
4239,51,3
5740,69,2
1936,36,4
14816,255,1
5417,274,3
7186,45,1


#### Define a `split_data` function for splitting data to training and test set

In [22]:
# We can define a function for this step as follows

def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [23]:
# lets try with both dummy table and scaled/normalized purchase table

train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

## 5. Baseline Model
Before running a more complicated approach such as collaborative filtering, we would like to use a baseline model to compare and evaluate models. Since baseline typically uses a very simple approach, techniques used beyond this approach should be chosen if they show relatively better accuracy and complexity.

### 5.1. Using a Popularity model as a baseline
* The popularity model takes the most popular items for recommendation. These items are products with the highest number of sells across customers.
* We use `turicreate` library for running and evaluating both baseline and collaborative filtering models below
* Training data is used for model selection

#### Using purchase counts

In [33]:
# variables to define field names
user_id = 'customerId'
item_id = 'productId'
target = 'purchase_count'
users_to_recommend = list(transactions[user_id])
n_rec = 10 # number of items to recommend
n_display = 30

In [34]:
popularity_model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 106868 observations with 23312 users and 300 items.

Data prepared in: 0.153818s

106868 observations to process; with 300 unique items.

In [35]:
# Get recommendations for a list of users to recommend (from customers file)
# Printed below is head / top 30 rows for first 3 customers with 10 recommendations each

popularity_recomm = popularity_model.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm.print_rows(n_display)

recommendations finished on 1000/62483 queries. users per second: 52432.9

recommendations finished on 2000/62483 queries. users per second: 74769.2

recommendations finished on 3000/62483 queries. users per second: 89341.6

recommendations finished on 4000/62483 queries. users per second: 100020

recommendations finished on 5000/62483 queries. users per second: 104162

...

recommendations finished on 59000/62483 queries. users per second: 122698

recommendations finished on 60000/62483 queries. users per second: 122810

recommendations finished on 61000/62483 queries. users per second: 123088

recommendations finished on 62000/62483 queries. users per second: 123084

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|     0      |    248    | 3.122448979591837  |  1   |
|     0      |    132    | 3.086206896551724  |  2   |
|     0      |     37    | 3.0505836575875485 |  3   |
|     0      |     0     | 2.9818355640535374 |  4   |
|     0      |     34    | 2.951219512195122  |  5   |
|     0      |    110    | 2.8819875776397517 |  6   |
|     0      |     3     | 2.847736625514403  |  7   |
|     0      |     27    | 2.8048780487804876 |  8   |
|     0      |     32    | 2.6809523809523808 |  9   |
|     0      |    230    | 2.612676056338028  |  10  |
|     1      |    248    | 3.122448979591837  |  1   |
|     1      |    132    | 3.086206896551724  |  2   |
|     1      |     37    | 3.0505836575875485 |  3   |
|     1      |     0     | 2.9818355640535374 |  4   |
|     1      |     34    | 2.951219512195122  |  5   |
|     1   

#### Define a `model` function for model selection

In [40]:
# Since turicreate is very accessible library, we can define a model selection function as below

def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [41]:
# variables to define field names
# constant variables include:
user_id = 'customerId'
item_id = 'productId'
users_to_recommend = list(customers[user_id])
n_rec = 10 # number of items to recommend
n_display = 30 # to print the head / first few rows in a defined dataset

#### Using purchase dummy

In [42]:
# these variables will change accordingly
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23296 users and 300 items.

Data prepared in: 0.143512s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 80638.7

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|    1553    |     87    |  1.0  |  1   |
|    1553    |    227    |  1.0  |  2   |
|    1553    |     44    |  1.0  |  3   |
|    1553    |     11    |  1.0  |  4   |
|    1553    |    131    |  1.0  |  5   |
|    1553    |    224    |  1.0  |  6   |
|    1553    |    289    |  1.0  |  7   |
|    1553    |    281    |  1.0  |  8   |
|    1553    |    275    |  1.0  |  9   |
|    1553    |    248    |  1.0  |  10  |
|   20400    |    227    |  1.0  |  1   |
|   20400    |     44    |  1.0  |  2   |
|   20400    |     11    |  1.0  |  3   |
|   20400    |     14    |  1.0  |  4   |
|   20400    |    131    |  1.0  |  5   |
|   20400    |    224    |  1.0  |  6   |
|   20400    |    289    |  1.0  |  7   |
|   20400    |    281    |  1.0  |  8   |
|   20400    |    275    |  1.0  |  9   |
|   20400    |    248    |  1.0  |  10  |
|   19750    |    227    |  1.0  |

#### Using normalized purchase count

In [43]:
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23342 users and 300 items.

Data prepared in: 0.13459s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 77411.4

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|    1553    |    226    |  0.7671232876712328 |  1   |
|    1553    |    230    |  0.3417910447761188 |  2   |
|    1553    |    247    |  0.3282828282828283 |  3   |
|    1553    |    125    | 0.26285714285714257 |  4   |
|    1553    |    294    | 0.25691056910569066 |  5   |
|    1553    |    155    | 0.24727272727272723 |  6   |
|    1553    |    248    |        0.235        |  7   |
|    1553    |     72    | 0.23183760683760685 |  8   |
|    1553    |    204    | 0.23116883116883113 |  9   |
|    1553    |    276    |  0.2261029411764706 |  10  |
|   20400    |    226    |  0.7671232876712328 |  1   |
|   20400    |    230    |  0.3417910447761188 |  2   |
|   20400    |    247    |  0.3282828282828283 |  3   |
|   20400    |    125    | 0.26285714285714257 |  4   |
|   20400    |    294    | 0.25691056910569066 |

#### Notes
* Once we created the model, we predicted the recommendation items using scores by popularity. As you can tell for each model results above, the rows show the first 30 records from 1000 users with 10 recommendations. These 30 records include 3 users and their recommended items, along with score and descending ranks. 
* In the result, although different models have different recommendation list, each user is recommended the same list of 10 items. This is because popularity is calculated by taking the most popular items across all users.
* If a grouping example below, products 132, 248, 37, and 34 are the most popular (best-selling) across customers. Using their purchase counts divided by the number of customers, we see that these products are at least bought 3 times on average in the training set of transactions (same as the first popularity measure on `purchase_count` variable)

In [ ]:
train.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending=False).head(20)

## 6. Collaborative Filtering Model

* In collaborative filtering, we would recommend items based on how similar users purchase items. For instance, if customer 1 and customer 2 bought similar items, e.g. 1 bought X, Y, Z and 2 bought X, Y, we would recommend an item Z to customer 2.

* To define similarity across users, we use the following steps:
    1. Create a user-item matrix, where index values represent unique customer IDs and column values represent unique product IDs
    
    2. Create an item-to-item similarity matrix. The idea is to calculate how similar a product is to another product. There are a number of ways of calculating this. In steps 6.1 and 6.2, we use cosine and pearson similarity measure, respectively.  
    
        * To calculate similarity between products X and Y, look at all customers who have rated both these items. For example, both X and Y have been rated by customers 1 and 2. 
        * We then create two item-vectors, v1 for item X and v2 for item Y, in the user-space of (1, 2) and then find the `cosine` or `pearson` angle/distance between these vectors. A zero angle or overlapping vectors with cosine value of 1 means total similarity (or per user, across all items, there is same rating) and an angle of 90 degree would mean cosine of 0 or no similarity.
        
    3. For each customer, we then predict his likelihood to buy a product (or his purchase counts) for products that he had not bought. 
    
        * For our example, we will calculate rating for user 2 in the case of item Z (target item). To calculate this we weigh the just-calculated similarity-measure between the target item and other items that customer has already bought. The weighing factor is the purchase counts given by the user to items already bought by him. 
        * We then scale this weighted sum with the sum of similarity-measures so that the calculated rating remains within a predefined limits. Thus, the predicted rating for item Z for user 2 would be calculated using similarity measures.

* While I wrote python scripts for all the process including finding similarity using python scripts (which can be found in `scripts` folder, we can use `turicreate` library for now to capture different measures like using `cosine` and `pearson` distance, and evaluate the best model.

### 6.1. `Cosine` similarity
* Similarity is the cosine of the angle between the 2 vectors of the item vectors of A and B
* It is defined by the following formula
![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTnRHSAx1c084UXF2wIHYwaHJLmq2qKtNk_YIv3RjHUO00xwlkt)
* Closer the vectors, smaller will be the angle and larger the cosine

#### Using purchase count

In [44]:
# these variables will change accordingly
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23312 users and 300 items.

Data prepared in: 0.121761s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.0605s

recommendations finished on 1000/1000 queries. users per second: 28808.5

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|    1553    |     2     | 0.11112298965454101  |  1   |
|    1553    |     35    | 0.07867621183395386  |  2   |
|    1553    |     5     | 0.07442752122879029  |  3   |
|    1553    |     41    | 0.06183123588562012  |  4   |
|    1553    |     17    | 0.05897554159164429  |  5   |
|    1553    |     21    | 0.05097031593322754  |  6   |
|    1553    |     33    | 0.04985777139663696  |  7   |
|    1553    |     47    | 0.04913526773452759  |  8   |
|    1553    |     61    | 0.04898881912231445  |  9   |
|    1553    |     15    | 0.04836616516113281  |  10  |
|   20400    |    280    | 0.09069609642028809  |  1   |
|   20400    |    284    | 0.043870747089385986 |  2   |
|   20400    |     42    | 0.04057210683822632  |  3   |
|   20400    |    267    | 0.03785592317581177  |  4   |
|   20400    |     26    | 0.03

#### Using purchase dummy

In [ ]:
# these variables will change accordingly
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

#### Using normalized purchase count

In [ ]:
name = 'cosine'
target = 'scaled_purchase_freq'
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

### 6.2. `Pearson` similarity
* Similarity is the pearson coefficient between the two vectors.
* It is defined by the following formula
![](http://critical-numbers.group.shef.ac.uk/glossary/images/correlationKT1.png)

#### Using purchase count

In [45]:
# these variables will change accordingly
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23312 users and 300 items.

Data prepared in: 0.114527s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.090032s

recommendations finished on 1000/1000 queries. users per second: 43827

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|    1553    |    248    | 3.121784340118875  |  1   |
|    1553    |    132    | 3.0862068965517246 |  2   |
|    1553    |     37    | 3.0505836575875476 |  3   |
|    1553    |     0     | 2.9791111311082847 |  4   |
|    1553    |     34    | 2.933261693686973  |  5   |
|    1553    |    110    | 2.873064631186658  |  6   |
|    1553    |     3     | 2.8477366255144028 |  7   |
|    1553    |     27    | 2.804878048780488  |  8   |
|    1553    |     32    | 2.6809523809523816 |  9   |
|    1553    |    230    | 2.6069272956378033 |  10  |
|   20400    |    248    | 3.1224489795918364 |  1   |
|   20400    |    132    | 3.075857384451505  |  2   |
|   20400    |     37    | 3.0505836575875476 |  3   |
|   20400    |     0     | 2.979850312150002  |  4   |
|   20400    |     34    | 2.9512195121951215 |  5   |
|   20400 

#### Using purchase dummy

In [46]:
# these variables will change accordingly
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23296 users and 300 items.

Data prepared in: 0.121808s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.061532s

recommendations finished on 1000/1000 queries. users per second: 32938.1

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|    1553    |     87    |  0.0  |  1   |
|    1553    |    227    |  0.0  |  2   |
|    1553    |     44    |  0.0  |  3   |
|    1553    |     11    |  0.0  |  4   |
|    1553    |    131    |  0.0  |  5   |
|    1553    |    224    |  0.0  |  6   |
|    1553    |    289    |  0.0  |  7   |
|    1553    |    281    |  0.0  |  8   |
|    1553    |    275    |  0.0  |  9   |
|    1553    |    248    |  0.0  |  10  |
|   20400    |    227    |  0.0  |  1   |
|   20400    |     44    |  0.0  |  2   |
|   20400    |     11    |  0.0  |  3   |
|   20400    |     14    |  0.0  |  4   |
|   20400    |    131    |  0.0  |  5   |
|   20400    |    224    |  0.0  |  6   |
|   20400    |    289    |  0.0  |  7   |
|   20400    |    281    |  0.0  |  8   |
|   20400    |    275    |  0.0  |  9   |
|   20400    |    248    |  0.0  |  10  |
|   19750    |    227    |  0.0  |

#### Using normalized purchase count

In [48]:
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23342 users and 300 items.

Data prepared in: 0.157504s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.151958s

recommendations finished on 1000/1000 queries. users per second: 46174.4

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|    1553    |    226    |  0.7671232876712331 |  1   |
|    1553    |    230    |  0.3415446242734566 |  2   |
|    1553    |    247    | 0.32813356335114935 |  3   |
|    1553    |    125    |  0.2625757940326417 |  4   |
|    1553    |    294    |  0.2569105691056912 |  5   |
|    1553    |    155    | 0.24727272727272728 |  6   |
|    1553    |    248    | 0.23449590861797334 |  7   |
|    1553    |     72    | 0.23155479769930884 |  8   |
|    1553    |    204    | 0.23116883116883113 |  9   |
|    1553    |    276    |  0.2256905962439144 |  10  |
|   20400    |    226    |  0.7671232876712331 |  1   |
|   20400    |    230    | 0.34179104477611927 |  2   |
|   20400    |    247    |  0.3282828282828281 |  3   |
|   20400    |    125    |  0.2621621526990617 |  4   |
|   20400    |    294    | 0.25636941853577544 |

#### Note
* In collaborative filtering above, we used two approaches: cosine and pearson distance. We also got to apply them to three training datasets with normal counts, dummy, or normalized counts of items purchase.
* We can see that the recommendations are different for each user. This suggests that personalization does exist. 
* But how good is this model compared to the baseline, and to each other? We need some means of evaluating a recommendation engine. Lets focus on that in the next section.

## 7. Model Evaluation
For evaluating recommendation engines, we can use the concept of precision-recall.

* RMSE (Root Mean Squared Errors)
    * Measures the error of predicted values
    * Lesser the RMSE value, better the recommendations
* Recall
    * What percentage of products that a user buys are actually recommended?
    * If a customer buys 5 products and the recommendation decided to show 3 of them, then the recall is 0.6
* Precision
    * Out of all the recommended items, how many the user actually liked?
    * If 5 products were recommended to the customer out of which he buys 4 of them, then precision is 0.8
    
* Why are both recall and precision important?
    * Consider a case where we recommend all products, so our customers will surely cover the items that they liked and bought. In this case, we have 100% recall! Does this mean our model is good?
    * We have to consider precision. If we recommend 300 items but user likes and buys only 3 of them, then precision is 0.1%! This very low precision indicates that the model is not great, despite their excellent recall.
    * So our aim has to be optimizing both recall and precision (to be close to 1 as possible).

Lets compare all the models we have built based on precision-recall characteristics:

In [52]:
# create initial callable variables

models_w_counts = [popularity_model, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]

names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

NameError: name 'cos_dummy' is not defined

#### Models on purchase counts

In [51]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)

NameError: name 'names_w_counts' is not defined

#### Models on purchase dummy

In [ ]:
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)

#### Models on normalized purchase frequency

In [ ]:
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

## 8. Model Selection
### 8.1. Evaluation summary
* Based on RMSE


    1. Popularity on purchase counts: 1.1111750034210488
    2. Cosine similarity on purchase counts: 1.9230643981653215
    3. Pearson similarity on purchase counts: 1.9231102838192284
    
    4. Popularity on purchase dummy: 0.9697374361161925
    5. Cosine similarity on purchase dummy: 0.9697509978436404
    6. Pearson similarity on purchase dummy: 0.9697745320187097
    
    7. Popularity on scaled purchase counts: 0.16230660626840343
    8. Cosine similarity on scaled purchase counts: 0.16229800354111104
    9. Pearson similarity on scaled purchase counts: 0.1622982668334026
    
* Based on Precision and Recall
![](../images/model_comparisons.png)


#### Notes

* Popularity v. Collaborative Filtering: We can see that the collaborative filtering algorithms work better than popularity model for purchase counts. Indeed, popularity model doesn’t give any personalizations as it only gives the same list of recommended items to every user.
* Precision and recall: Looking at the summary above, we see that the precision and recall for Purchase Counts > Purchase Dummy > Normalized Purchase Counts. However, because the recommendation scores for the normalized purchase data is zero and constant, we choose the dummy. In fact, the RMSE isn’t much different between models on the dummy and those on the normalized data.
* RMSE: Since RMSE is higher using pearson distance thancosine, we would choose model the smaller mean squared errors, which in this case would be cosine.
Therefore, we select the Cosine similarity on Purchase Dummy approach as our final model.

## 8. Final Output
* In this step, we would like to manipulate format for recommendation output to one we can export to csv, and also a function that will return recommendation list given a customer ID.
* We need to first rerun the model using the whole dataset, as we came to a final model using train data and evaluated with test set.

In [ ]:
users_to_recommend = list(customers[user_id])


final_model = tc.item_similarity_recommender.create(tc.SFrame(data_dummy), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='purchase_dummy', 
                                            similarity_type='cosine')

recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

### 8.1. CSV output file

In [ ]:
df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head()

In [ ]:
df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id].transform(lambda x: '|'.join(x.astype(str)))
df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates().sort_values('customerId').set_index('customerId')

#### Define a function to create a desired output

In [ ]:
def create_output(model, users_to_recommend, n_rec, print_csv=True):
    recomendation = model.recommend(users=users_to_recommend, k=n_rec)
    df_rec = recomendation.to_dataframe()
    df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))
    df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates() \
        .sort_values('customerId').set_index('customerId')
    if print_csv:
        df_output.to_csv('../output/option1_recommendation.csv')
        print("An output file can be found in 'output' folder with name 'option1_recommendation.csv'")
    return df_output

In [ ]:
df_output = create_output(pear_norm, users_to_recommend, n_rec, print_csv=True)
print(df_output.shape)
df_output.head()

### 8.2. Customer recommendation function

In [ ]:
def customer_recomendation(customer_id):
    if customer_id not in df_output.index:
        print('Customer not found.')
        return customer_id
    return df_output.loc[customer_id]

In [ ]:
customer_recomendation(4)

In [ ]:
customer_recomendation(21)

## Summary
In this exercise, we were able to traverse a step-by-step process for making recommendations to customers. We used Collaborative Filtering approaches with `cosine` and `pearson` measure and compare the models with our baseline popularity model. We also prepared three sets of data that include regular buying count, buying dummy, as well as normalized purchase frequency as our target variable. Using RMSE, precision and recall, we evaluated our models and observed the impact of personalization. Finally, we selected the Cosine approach in dummy purchase data. 